---
## **NOTA:** Toda la información producida durante el proyecto esta disponible en **https://drive.google.com/drive/folders/1v6ttK1kLlbDPv3gwIl_qY6QtO9HhZjMk?usp=sharing** el drive de Google: las bases de datos con los tweets descargados y procesados se encuentran como **archivos CSV** zipeados, asi como tambien los modelos de NLP (serializados con pickle).
---
#   **TA_POLITICA-02_twitterscrapping.ipynb**
>## ENTRADA:
>## **"politicosdb.csv"** : ( drive: **politicos_lista_anotada.zip** )
>## producida por el notebook anterior, almacena cantidad de tweets recuperados hasta el momento, asi como la fecha del último tweet recuperado, es como el log de avance de GetOldTweets3 para cada vez que se lo llama.
---
>## SALIDA:
>## **"tweetsdb_@nombre_usario.csv"**  : ( drive: **tweets_politicos_db.zip** ) todos los tweets por cada @nombre_usuario


In [1]:
from platform import python_version
print(python_version())

3.7.7


In [2]:
import pandas as pd
import GetOldTweets3 as got
import tweepy
import twitter
import json
import re
import sys
import time
import random
import pickle
from datetime import datetime,timedelta, date
sys.version

'3.7.7 (default, Apr 15 2020, 05:09:04) [MSC v.1916 64 bit (AMD64)]'

In [3]:
# Cargara las credenciales 
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

>### **"politicosdb.csv"** información sobre cada cuenta , aqui utilizamos la **fecha_de_creacion** y **numero_de_tweets**. 

In [4]:
db_politicos_csv = "politicosdb.csv"

In [5]:
api = twitter.Api(consumer_key=creds['CONSUMER_KEY'],
                  consumer_secret=creds['CONSUMER_SECRET'] ,
                  access_token_key=creds['ACCESS_TOKEN'],
                  access_token_secret=creds['ACCESS_SECRET'])

In [6]:
auth = tweepy.OAuthHandler(consumer_key=creds['CONSUMER_KEY'], consumer_secret=creds['CONSUMER_SECRET'])
auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
apiTweepy = tweepy.API(auth)

>### con **GetOldTweets3** recogemos los tweets desde la fecha_de_creacion [creado] hasta donde se quedo tweepy
>### No se pueden recuperar Retweets, por lo que el programa avisa cuando se recuperan (casi <100%) todos los tweets.

In [15]:
def actualizarPoliticosDB_tweets():
    politicosdb_df = pd.read_csv(db_politicos_csv )

    for idx,scr_name in enumerate(politicosdb_df['screen_name']):   
        usuario_df = pd.read_csv("tweetsdb_"+scr_name+".csv" )
        politicosdb_df.loc[idx,'tweets']=len(usuario_df)
        total=politicosdb_df.loc[idx,'total_tweets']
        scr_name=politicosdb_df.loc[idx,'screen_name']
        porcentaje=100*politicosdb_df.loc[idx,'tweets']/total
        if(len(scr_name)>13):
            tab="\t"
        else:
            tab="\t\t"
        print("Usuario:",scr_name,tab,f"({(porcentaje):.2f}%)")

    politicosdb_df.to_csv(db_politicos_csv , index=False)
    return politicosdb_df
    

pdf = actualizarPoliticosDB_tweets()
pdf

Usuario: @GeneralDelAire1 	 (16.40%)
Usuario: @EnriqueFCHA 		 (60.16%)
Usuario: @patriaroja 		 (28.24%)
Usuario: @SegundoMendezA 	 (56.64%)
Usuario: @ManuelDammert 	 (69.50%)
Usuario: @IndiraHuilca 		 (60.54%)
Usuario: @MarisaGlave 		 (58.36%)
Usuario: @GGG_pe 		 (99.37%)
Usuario: @LuciaAlvites 		 (89.33%)
Usuario: @VicAZeballos 		 (97.26%)
Usuario: @jtownsenddc 		 (98.33%)
Usuario: @MirtyVas 		 (99.87%)
Usuario: @Vero_Mendoza_F 	 (71.14%)
Usuario: @vozdelatierra 	 (36.41%)
Usuario: @_zavandija 		 (19.42%)
Usuario: @AntauroHumala 	 (100.00%)
Usuario: @arlettecontrers 	 (97.99%)
Usuario: @anajarav 		 (38.28%)
Usuario: @DianaMiloslavic 	 (97.50%)
Usuario: @sigridbazan 		 (42.37%)
Usuario: @RichardArcePeru 	 (54.28%)
Usuario: @suselparedes 		 (8.68%)
Usuario: @EdgarOchoaPezo 	 (99.93%)
Usuario: @HumbertoTYL 		 (86.96%)
Usuario: @HCevallosFlores 	 (99.00%)
Usuario: @rogelio_tucto 	 (99.81%)
Usuario: @FrenteAmplioPe 	 (50.76%)
Usuario: @arturoayala_4 	 (99.83%)
Usuario: @LucianoRevoredo 	 (

,id,screen_name,nombre,partido,nombre_partido,orientacion,total_tweets,tweets,fecha_creado,fecha_min,descripcion,num_seguidores
0,1186714680527593474,@GeneralDelAire1,General del Aire - El Bloqueado - #GeneralDeTodos,nul,no conocido,nul,19676,3227,2019-10-22,2019-10-11,"Ayudaré a todos.\n\nSi quieren ignorar la ley,...",9033
1,1017875140313518081,@EnriqueFCHA,Enrique Fernández Chacón,fa,frente amplio,izq,5233,3148,2018-07-13,2018-07-09,Político de toda la vida 🇵🇪\n#CuarentenaSinHam...,20145
2,102238423,@patriaroja,Patria Roja,pr,patraia roja,izq,14998,4235,2010-01-06,2012-06-03,Partido Comunista del Perú fundado por José Ca...,4116
3,1058118893133316096,@SegundoMendezA,Segundo Mendez,nul,no conocido,izq,15014,8504,2018-11-01,2018-08-31,Reformista y en contra de la cleptocracia. Nue...,2364
4,1436400703,@ManuelDammert,Manuel Dammert,np,nuevo peru,izq,6629,4607,2013-05-17,2012-05-04,Porque mi patria es hermosa la defiendo con mi...,26520
...,...,...,...,...,...,...,...,...,...,...,...,...
97,125040813,@geovanniacate,Geovanni Acate,app,alianza para el progreso,cder,11224,7705,2010-03-21,2010-02-21,La relación con los políticos es puramente ins...,1394
98,52888826,@FabioTr_,Fabio Trujillo,ap,accion popular,cder,2712,2678,2009-07-01,2009-05-04,Candidato con #15 al Congreso de la República ...,257
99,627196536,@AleGarciaOviedo,Alejandra García Oviedo,ap,accion popular,cder,4875,3532,2012-07-05,2012-06-22,27 años. Regidora de Lima 2019-2022. President...,227
100,869665128534478852,@MadeleineOster3,Madeleine Osterling,nul,no conocido,cder,20429,10569,2017-05-30,2017-04-26,"Abogada en ejercicio, profesora universitaria",36564


## Usamos **GetOldTweets3** para extraer todos los tweets hasta la creación de cada cuenta. 
>## NOTA: Los Retweets no se recuperan.

In [17]:
#funcion que nos permite crear un rango de fechas datetime
def daterange(start_date, end_date ,dias=30): 
    if start_date < end_date:
        #FORWARD: start_date < end_date
        for n in range(0,int((end_date - start_date).days),dias):
            yield start_date + timedelta(n)
    else:
        #BACKWARD: start_date > end_date
        for n in range(0,int((start_date - end_date).days),dias):
            yield start_date - timedelta(n)
            

In [35]:
politicosdb_df = pd.read_csv(db_politicos_csv )
MAXTWEETS_QUERY = 250
DIAS_TRAMO = 300

def recuperarOldTweets():
    global usuario_df
    tweetCriteria = got.manager.TweetCriteria().setUsername(scr_name)\
                                               .setSince(to_date_str)\
                                               .setUntil(from_date_str)\
                                               .setMaxTweets(MAXTWEETS_QUERY)\
                                               .setEmoji("unicode")
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)

    old_tweets = [[tw.id, 
            tw.username,
            tw.text,
            datetime.strptime(tw.formatted_date, '%a %b %d %H:%M:%S +%G %Y').strftime("%Y-%m-%d"),
            tw.retweets,
            tw.mentions,
            tw.hashtags] for tw in tweets]

    news_df = pd.DataFrame(old_tweets, 
              columns = ['id','User', 'Text','Date','Retweets', 'Mentions', 'HashTags'])
    usuario_df = usuario_df.append(news_df)
    print(f"Tweets recuperados->{len(tweets):04d} desde {from_date_str} hasta {to_date_str}. Tweets hasta ahora:{len(usuario_df)}. {dias_recuperados}/{DIAS_TRAMO}")
    
# DIAS_TRAMO : la cantidad de días antes de ultimo de los cuales vamos a extraer tweets
# dias_periodo : periodo del cual se extraen aprox. max=100 tweets
# dias_periodo = 10
# tweets_dia = tweets_total / dias_en_twitter


dt_hoy = datetime.today()

for idx,scr_name in enumerate(politicosdb_df['screen_name']):   # pruebas : enumerate(['@MarthaChavezC']): #
    
    print("Descansando")
    time.sleep(3)
    scr_name= scr_name[1:]
    #cargamos el dataframe que va a ser extendido con old tweets de 'scr_name'
    usuario_df = pd.read_csv("tweetsdb_@"+scr_name+".csv" )
    
    nombre = politicosdb_df['nombre'].iloc[idx]
    partido = politicosdb_df['partido'].iloc[idx]
    fecha_min_str = politicosdb_df['fecha_min'].iloc[idx]    
    fecha_creado_str = politicosdb_df['fecha_creado'].iloc[idx]
    fecha_min_dt = datetime.strptime(fecha_min_str, '%Y-%m-%d')
    fecha_creado_dt = datetime.strptime(fecha_creado_str,"%Y-%m-%d")
    fecha_since_dt = fecha_min_dt - timedelta(DIAS_TRAMO)  
#    if fecha_since_dt < fecha_creado_dt:
 #       fecha_since_dt = fecha_creado_dt
    total_tweets = politicosdb_df['total_tweets'].iloc[idx]
    total_tweets_recuperados = politicosdb_df['tweets'].iloc[idx]
    
    dias_en_twitter = (dt_hoy-datetime.strptime(politicosdb_df.iloc[idx].fecha_creado,"%Y-%m-%d")).days
    tweets_dia = total_tweets / dias_en_twitter
    dias_periodo = int((0.84*MAXTWEETS_QUERY)/tweets_dia)+1
    if dias_periodo > DIAS_TRAMO:
        dias_periodo = DIAS_TRAMO
    dias_recuperados = 0
    
    print(f"Usuario:{scr_name} ({nombre})/{partido}/ Creado:{fecha_creado_str} / Fecha_min:{fecha_min_str} / Total:{total_tweets} Recuperados:{total_tweets_recuperados} / Tramo:{DIAS_TRAMO}@{dias_periodo}")
    if total_tweets-total_tweets_recuperados < 10 or fecha_min_dt <= fecha_creado_dt:
        print("------Se han recuperado (casi) todos lo tweets de este usuario-------")
        continue    
    
    start_date, end_date = fecha_min_dt, fecha_since_dt
    end_date_str = end_date.strftime("%Y-%m-%d")
    daterange_generator=daterange(start_date, end_date, dias_periodo)
    from_date=next(daterange_generator)
    from_date_str=from_date.strftime("%Y-%m-%d")    
    
    i = 0
    for single_date in daterange_generator:
        to_date = single_date + timedelta(1)
        to_date_str=to_date.strftime("%Y-%m-%d")
        
        dias_recuperados += dias_periodo
        recuperarOldTweets()
        time.sleep(2)
        
        from_date = single_date    
        from_date_str=from_date.strftime("%Y-%m-%d")
       
    #el ultimo periodo
    to_date_str=end_date_str
    dias_recuperados += dias_periodo
    recuperarOldTweets()
   
    usuario_df.to_csv("tweetsdb_@"+scr_name+".csv", index=False)
     
    politicosdb_df.loc[idx,'tweets']=len(usuario_df)
    politicosdb_df.loc[idx,'fecha_min']=end_date_str
    politicosdb_df.to_csv(db_politicos_csv , index=False)

Descansando
Usuario:GeneralDelAire1 (General del Aire - El Bloqueado - #GeneralDeTodos)/nul/ Creado:2019-10-22 / Fecha_min:2019-10-11 / Total:19676 Recuperados:3227 / Tramo:300@3
------Se han recuperado (casi) todos lo tweets de este usuario-------
Descansando
Usuario:EnriqueFCHA (Enrique Fernández Chacón)/fa/ Creado:2018-07-13 / Fecha_min:2018-07-09 / Total:5233 Recuperados:3148 / Tramo:300@29
------Se han recuperado (casi) todos lo tweets de este usuario-------
Descansando
Usuario:patriaroja (Patria Roja)/pr/ Creado:2010-01-06 / Fecha_min:2009-09-07 / Total:14998 Recuperados:4295 / Tramo:300@54
------Se han recuperado (casi) todos lo tweets de este usuario-------
Descansando
Usuario:SegundoMendezA (Segundo Mendez)/nul/ Creado:2018-11-01 / Fecha_min:2018-08-31 / Total:15014 Recuperados:8504 / Tramo:300@9
------Se han recuperado (casi) todos lo tweets de este usuario-------
Descansando
Usuario:ManuelDammert (Manuel Dammert)/np/ Creado:2013-05-17 / Fecha_min:2012-05-04 / Total:6629 Recu

---
## El trabajo continua en el notebook   **TA_POLITICA-03_preprocessing.ipynb**
